In [ ]:
# The purpose of this code is to develop a GIF with a map of Minnesota (MN) successively adding MN state highways-just for fun

In [1]:
import pandas as pd
import geopandas as gpd
import descartes
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry
import imageio # to make the GIF file
import os
from progressbar import ProgressBar
import warnings

In [2]:
roads = 'STREETS_LOAD.shp'

In [3]:
states = '/Users/matthewwilliamhaas/Documents/shapefile_data_sources/cb_2018_us_state_20m/cb_2018_us_state_20m.shp'
map_states = gpd.read_file(states)

In [4]:
map_states[map_states.STATEFP == '27']

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
28,27,00662849,0400000US27,27,MN,Minnesota,00,206228939448,18945217189,"POLYGON ((-97.22904 49.00069, -96.93096 48.999..."


In [5]:
MN = map_states[(map_states.STATEFP == '27')]
MN = MN.to_crs({'init' : 'epsg:26915'})

In [6]:
# read in data
road_data = gpd.read_file(roads)

In [25]:
# Convert code for making one-off highway maps to a function to do in a high-throughput manner.
# Needed arguments: 1) path to image (highway sign), 2) indicator of current highway to plot geodata, and 3) output filename

# Initiate empty list to hold highways that have already been plotted
#alreadyPlotted = [] # Keep outside function definitation so it doesn't get cleared every time the function is called
#geoDataAll = gpd.GeoSeries(crs='epsg:26915') # make empty geopandas geoseries dataframe

def MakeHwyMap (imagefile, currentHwy, outfile):
    global geoDataAll
    data = road_data.STREET_NAM == currentHwy
    geoData = road_data[(data)].geometry
    im = plt.imread(imagefile)
    fig, ax = plt.subplots(1, figsize = (20,15))
    MN.plot(color = 'white', linewidth = 1.0, ax = ax, edgecolor = 'black')
    plt.title('Minnesota state ("trunk") highway system', fontsize = 60)
    plt.annotate('Source: U.S. Census Bureau (map); Minnesota Department of Transportation (roads)', (0,0), (0,-10), xycoords = 'axes fraction', textcoords = 'offset points', va = 'top', ha = 'left', fontsize = 20)
    #if currentHwy in alreadyPlotted:
    geoDataAll.plot(color ='grey', linewidth = 3.0, ax = ax, edgecolor = 'black')
    #else:
    geoData.plot(color ='red', linewidth = 3.0, ax = ax, edgecolor = 'black')
    #alreadyPlotted.append(currentHwy)
    geoDataAll = geoDataAll.append(geoData)
    newax = fig.add_axes([0.8, 0.5, 0.2, 0.3], anchor = 'NE', zorder = 1)
    newax.imshow(im)
    ax.axis('off') # turn off the axis
    newax.axis('off')
    plt.savefig(outfile)
    plt.close('all')

In [35]:
infile = open('minnesota_highway_list.csv', 'r', encoding='utf-8-sig') # open file
highways = infile.readlines()  # begin reading data

out_PNG_list = [] # make empty list to hold names of output files to make GIF at end
#alreadyPlotted = []
geoDataAll = gpd.GeoSeries(crs='epsg:26915') # make empty geopandas geoseries dataframe

pbar = ProgressBar()

# Run the MakeHwyMap function for each line in the file (each line represents a highway)
for line in pbar(highways):
    line = line.rstrip()
    line = line.split(',')
    warnings.filterwarnings('ignore') # silence warnings
    MakeHwyMap(imagefile = line[0], currentHwy = line[1], outfile = line[2])
    out_PNG_list.append(line[2])

100% |########################################################################|


In [36]:
# make the GIF file
with imageio.get_writer('mn_state_highways.gif', mode = 'I', duration = 1) as writer:
    for filename in out_PNG_list:
        image = imageio.imread(filename)
        writer.append_data(image)